In [2]:
import numpy as np
import pandas as pd
import networkx as nx
import torch
import torch.nn as nn
from tqdm import tqdm, trange
import matplotlib.pyplot as plt

In [33]:
from data_loader import load_data, load_npz, load_random
from LPA import LPA
from utils import *

In [4]:
dataset = 'R8'

In [5]:
adj, adj_n, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, train_size, test_size = load_corpus(dataset)

In [6]:
y_train.sum()

4937.0

In [7]:
train_size = int(y_train.sum())
val_size = int(y_val.sum())

vocab_size = adj.shape[0] - train_size - val_size - test_size

In [8]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(torch.cuda.get_device_name(0))
else:
    device = torch.device('cpu')
    print("CPU")

NVIDIA GeForce RTX 3060 Laptop GPU


In [9]:
adj_final = torch.load(dataset + '_adj_final.pt', map_location=device)

In [10]:
epochs = 10

In [11]:
def get_mask(idx, length):
    mask = np.zeros(length)
    mask[idx] = 1
    return np.array(mask, dtype=np.float64)

In [12]:
rows = np.concatenate([np.arange(train_size + val_size), np.arange(-test_size, 0)])
rows.shape

(7674,)

In [13]:
cols = rows.copy()
cols.shape

(7674,)

In [14]:
adj_subset = adj_final[rows][:, cols]
# adj_subset = adj_final

In [15]:
adj_subset = adj_subset - torch.diag(torch.diag(adj_subset))

In [16]:
# torch.diag(torch.diag(adj_subset))

In [17]:
adj_subset

tensor([[0.0000e+00, 1.9356e+02, 2.4833e+01,  ..., 3.0130e+00, 1.3880e+01,
         7.3485e-03],
        [1.9356e+02, 0.0000e+00, 2.3631e+02,  ..., 6.4734e+01, 2.4176e+01,
         5.9222e+00],
        [2.4833e+01, 2.3631e+02, 0.0000e+00,  ..., 1.3724e+01, 0.0000e+00,
         7.3485e-03],
        ...,
        [3.0130e+00, 6.4734e+01, 1.3724e+01,  ..., 0.0000e+00, 0.0000e+00,
         2.3954e+01],
        [1.3880e+01, 2.4176e+01, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [7.3485e-03, 5.9222e+00, 7.3485e-03,  ..., 2.3954e+01, 0.0000e+00,
         0.0000e+00]], device='cuda:0', dtype=torch.float64)

In [144]:
adj_subset = adj_subset / torch.sum(adj_subset, dim=1, keepdim=True)

In [18]:
adj_subset - adj_subset.T

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float64)

In [119]:
np.max(adj_n.A)

176.06772860094713

In [19]:
adj_subset

tensor([[0.0000e+00, 1.9356e+02, 2.4833e+01,  ..., 3.0130e+00, 1.3880e+01,
         7.3485e-03],
        [1.9356e+02, 0.0000e+00, 2.3631e+02,  ..., 6.4734e+01, 2.4176e+01,
         5.9222e+00],
        [2.4833e+01, 2.3631e+02, 0.0000e+00,  ..., 1.3724e+01, 0.0000e+00,
         7.3485e-03],
        ...,
        [3.0130e+00, 6.4734e+01, 1.3724e+01,  ..., 0.0000e+00, 0.0000e+00,
         2.3954e+01],
        [1.3880e+01, 2.4176e+01, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [7.3485e-03, 5.9222e+00, 7.3485e-03,  ..., 2.3954e+01, 0.0000e+00,
         0.0000e+00]], device='cuda:0', dtype=torch.float64)

In [20]:
adj_rand = np.random.rand(adj_subset.shape[0], adj_subset.shape[0])
adj_rand

array([[0.78475308, 0.02345429, 0.95095028, ..., 0.22065812, 0.24702502,
        0.24430901],
       [0.21838327, 0.32350155, 0.56483935, ..., 0.26789303, 0.96283899,
        0.90467141],
       [0.44932207, 0.9973103 , 0.40523261, ..., 0.97192255, 0.19665006,
        0.92915418],
       ...,
       [0.10076742, 0.30328208, 0.02530284, ..., 0.95864615, 0.42344195,
        0.09259884],
       [0.97906408, 0.63835324, 0.94200984, ..., 0.33621052, 0.74593298,
        0.49963645],
       [0.32079951, 0.39525113, 0.62779704, ..., 0.67962962, 0.43407657,
        0.2695939 ]])

In [154]:
adj_rand = torch.tensor(adj_rand) - torch.diag(torch.diag(torch.tensor(adj_rand)))
adj_rand = adj_rand / torch.sum(adj_rand, dim=1, keepdim=True)
adj_rand

tensor([[0.0000e+00, 1.1853e-04, 2.0053e-04,  ..., 2.3773e-04, 4.0318e-05,
         5.8116e-05],
        [1.8709e-04, 0.0000e+00, 3.4282e-05,  ..., 1.5275e-04, 1.0604e-04,
         2.3928e-04],
        [2.4825e-04, 1.8632e-04, 0.0000e+00,  ..., 2.7557e-07, 1.6056e-04,
         1.5020e-04],
        ...,
        [1.2649e-05, 1.9589e-04, 1.2863e-05,  ..., 0.0000e+00, 1.5135e-04,
         1.3827e-04],
        [6.2305e-05, 2.6294e-06, 1.0366e-04,  ..., 8.8925e-05, 0.0000e+00,
         1.9198e-04],
        [2.2914e-04, 2.5274e-04, 1.1023e-05,  ..., 1.1512e-04, 7.5519e-05,
         0.0000e+00]], dtype=torch.float64)

In [34]:
model = LPA(adj_subset)

c:\Users\Shaz\LPA-node-classification\torch_src\LPA.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.adj = torch.tensor(adj, dtype=torch.float64, requires_grad=True)


In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e+6)

In [36]:
all_train = torch.tensor(y_train[rows], dtype=torch.float64).to(device)
val_input = torch.tensor(y_train[rows], dtype=torch.float64).to(device)
test_input = torch.tensor(y_train[rows] + y_val[rows], dtype=torch.float64).to(device)

In [37]:
train_labels = torch.argmax(torch.tensor(y_train[rows]), dim=1).to(device)
val_labels = torch.argmax(torch.tensor(y_val[rows]), dim=1).to(device)
test_labels = torch.argmax(torch.tensor(y_test[rows]), dim=1).to(device)

In [38]:
type(train_mask)

torch.Tensor

In [39]:
for epoch in range(epochs):
    model.train()
    # print(model)
    # for p in model.parameters():
    #     print(p.name, p.data, p.requires_grad)
    train_indices = np.random.choice(list(range(train_size)), size=int(train_size * 0.8), replace=False)
    train_mask = torch.tensor(get_mask(train_indices, adj_subset.shape[0])).to(device)
    train_input = all_train * train_mask[:, None]
    print(f'\nEpoch {epoch}: ')
    outputs = model(train_input)
    loss = criterion(outputs * torch.tensor(train_mask[:, None]), train_labels * torch.tensor(train_mask, dtype=torch.int32))
    # train_acc = np.sum(torch.argmax(outputs, dim=1) == train_target)

    preds = torch.argmax(outputs, dim=1)
    train_acc = torch.eq(preds[:train_size], torch.tensor(train_labels[:train_size])).sum() / train_size
    print(f'Training Loss: {loss}\tTraining Accuracy: {train_acc}')

    optimizer.zero_grad()
    # print(model.lpa_layer.adj.grad)
    loss.backward()
    # plt.hist(model.lpa_layer.adj.grad)
    # plt.show()
    # print(model.lpa_layer.adj.grad)
    optimizer.step()

    model.eval()
    preds = model(val_input)
    loss = criterion(preds[train_size: train_size+val_size], val_labels[train_size: train_size+val_size])
    preds = torch.argmax(preds, dim=1)
    val_acc = torch.eq(preds[train_size: train_size+val_size], val_labels[train_size: train_size+val_size]).sum() / val_size
    print(f'Validation Loss: {loss}\tValidation Accuracy: {val_acc}')


Epoch 0: 
tensor(1.9365e+09, device='cuda:0', dtype=torch.float64,
       grad_fn=<SumBackward0>)
Training Loss: 1.7379766143615853	Training Accuracy: 0.8531496524810791


C:\Users\Shaz\AppData\Local\Temp/ipykernel_21940/2680099027.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = criterion(outputs * torch.tensor(train_mask[:, None]), train_labels * torch.tensor(train_mask, dtype=torch.int32))
C:\Users\Shaz\AppData\Local\Temp/ipykernel_21940/2680099027.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_acc = torch.eq(preds[:train_size], torch.tensor(train_labels[:train_size])).sum() / train_size


tensor(49826603.0068, device='cuda:0', dtype=torch.float64,
       grad_fn=<SumBackward0>)
Validation Loss: 1.403570880023469	Validation Accuracy: 0.8704379200935364

Epoch 1: 
tensor(49826603.0068, device='cuda:0', dtype=torch.float64,
       grad_fn=<SumBackward0>)
Training Loss: 1.7406801886021412	Training Accuracy: 0.853959858417511
tensor(-1.8345e+09, device='cuda:0', dtype=torch.float64,
       grad_fn=<SumBackward0>)
Validation Loss: 1.403570880023469	Validation Accuracy: 0.8704379200935364

Epoch 2: 
tensor(-1.8345e+09, device='cuda:0', dtype=torch.float64,
       grad_fn=<SumBackward0>)
Training Loss: 1.7384650295831454	Training Accuracy: 0.8604415655136108
tensor(-3.2765e+09, device='cuda:0', dtype=torch.float64,
       grad_fn=<SumBackward0>)
Validation Loss: 1.403570880023469	Validation Accuracy: 0.8704379200935364

Epoch 3: 
tensor(-3.2765e+09, device='cuda:0', dtype=torch.float64,
       grad_fn=<SumBackward0>)
Training Loss: 1.7378134792186613	Training Accuracy: 0.855985

In [143]:
print(torch.max(model.lpa_layer.adj.grad))

tensor(4.6505e-06, dtype=torch.float64)


In [163]:
test_preds = model(test_input)
test_preds = torch.argmax(test_preds, dim=1)
test_acc = torch.eq(test_preds[-1 * test_size:], test_labels[-1 * test_size:]).sum() / test_size
print("Test Accuracy: ", test_acc)

Test Accuracy:  tensor(0.4947)
